# Solving BLP Homework

This notebook implements the solution to the BLP homework assignment using Python and pyBLP.

## Overview

We will:
- Load the fake data
- Compute true elasticities and diversion ratios
- Estimate mis-specified models (OLS logit, IV logit, nested logit)
- Estimate the correctly specified BLP model
- Perform merger simulations with and without efficiencies
- Compute welfare impacts

In [1]:
import numpy as np
import pandas as pd
import pyblp
import scipy.optimize as opt
from scipy.special import logsumexp
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

In [ ]:
# Load data
# Instead of loading from CSV, generate the data in Python

import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal

# Model parameters
T = 600  # Number of markets
J = 4    # Number of products per market
N_sim = 200  # Number of simulation draws

# Demand parameters
beta_1 = 1        # Quality coefficient
alpha = -2        # Price coefficient
beta_2_mean = 4   # Satellite mean preference
beta_3_mean = 4   # Wired mean preference
sigma_2 = 1       # Satellite preference std dev
sigma_3 = 1       # Wired preference std dev

# Supply parameters
gamma_0 = 0.5     # Cost intercept
gamma_1 = 0.25    # Cost shifter coefficient

np.random.seed(42)

# Generate market and product identifiers
market_ids = np.repeat(np.arange(1, T+1), J)
product_ids = np.tile(np.arange(1, J+1), T)
firm_ids = np.tile(np.arange(1, J+1), T)

# Product characteristics (satellite/wired dummies)
satellite = np.tile([1, 1, 0, 0], T)
wired = np.tile([0, 0, 1, 1], T)

# Observable characteristics
x = np.abs(np.random.randn(T * J))  # Quality characteristic
w = np.abs(np.random.randn(T * J))  # Cost shifter

# Correlated demand and cost unobservables
xi_omega_cov = np.array([[1.0, 0.25], [0.25, 1.0]])
xi_omega = multivariate_normal.rvs(mean=[0, 0], cov=xi_omega_cov, size=T*J)
xi = xi_omega[:, 0]     # Demand unobservable
omega = xi_omega[:, 1]  # Cost unobservable

# Log marginal cost function
log_mc = gamma_0 + gamma_1 * w + omega / 8
marginal_costs = np.exp(log_mc)

# Function to compute shares and derivatives
def blp_shares_and_derivatives(prices, x, xi, satellite, wired, params):
    J = len(prices)
    shares_sum = np.zeros(J)
    dsdp_sum = np.zeros((J, J))
    
    for i in range(params['N_sim']):
        beta_2 = np.random.normal(params['beta_2_mean'], params['sigma_2'])
        beta_3 = np.random.normal(params['beta_3_mean'], params['sigma_3'])
        V = params['beta_1'] * x + params['alpha'] * prices + beta_2 * satellite + beta_3 * wired + xi
        V_max = np.max(V)
        exp_V = np.exp(V - V_max)
        exp_0 = np.exp(-V_max)
        denom = exp_0 + np.sum(exp_V)
        s_i = exp_V / denom
        shares_sum += s_i
        for j in range(J):
            for k in range(J):
                if j == k:
                    dsdp_sum[j, k] += params['alpha'] * s_i[j] * (1 - s_i[j])
                else:
                    dsdp_sum[j, k] += -params['alpha'] * s_i[j] * s_i[k]
    shares = shares_sum / params['N_sim']
    dsdp = dsdp_sum / params['N_sim']
    return shares, dsdp

# Function to compute FOC residuals
def compute_foc_residuals(prices, x, xi, satellite, wired, mc, params):
    shares, dsdp = blp_shares_and_derivatives(prices, x, xi, satellite, wired, params)
    residuals = np.zeros(J)
    for j in range(J):
        markup = prices[j] - mc[j]
        residuals[j] = markup * dsdp[j, j] + shares[j]
    return residuals

# Solve for equilibrium prices
equilibrium_prices = np.zeros(T * J)
params = {'beta_1': beta_1, 'alpha': alpha, 'beta_2_mean': beta_2_mean, 'beta_3_mean': beta_3_mean, 'sigma_2': sigma_2, 'sigma_3': sigma_3, 'N_sim': N_sim}

for t in range(T):
    start_idx = t * J
    end_idx = (t + 1) * J
    indices = slice(start_idx, end_idx)
    
    x_t = x[indices]
    xi_t = xi[indices]
    satellite_t = satellite[indices]
    wired_t = wired[indices]
    mc_t = marginal_costs[indices]
    
    # Solve FOC system
    def foc_system(p):
        return compute_foc_residuals(p, x_t, xi_t, satellite_t, wired_t, mc_t, params)
    
    p_eq = fsolve(foc_system, mc_t + 0.5)
    equilibrium_prices[indices] = p_eq

# Calculate observed market shares
observed_shares = np.zeros(T * J)
for t in range(T):
    start_idx = t * J
    end_idx = (t + 1) * J
    indices = slice(start_idx, end_idx)
    
    prices_t = equilibrium_prices[indices]
    x_t = x[indices]
    xi_t = xi[indices]
    satellite_t = satellite[indices]
    wired_t = wired[indices]
    
    shares_t, _ = blp_shares_and_derivatives(prices_t, x_t, xi_t, satellite_t, wired_t, params)
    observed_shares[indices] = shares_t

# Create DataFrame
data = pd.DataFrame({
    'market_ids': market_ids,
    'product_ids': product_ids,
    'firm_ids': firm_ids,
    'shares': observed_shares,
    'prices': equilibrium_prices,
    'x': x,
    'w': w,
    'xi': xi,
    'omega': omega,
    'satellite': satellite,
    'wired': wired,
    'marginal_cost': marginal_costs
})

print("Data generated, shape:", data.shape)

# True parameters
T, J = 600, 4
true_params = {
    'alpha': -2,
    'beta1': 1,
    'beta2_mean': 4,
    'sigma2': 1,
    'beta3_mean': 4,
    'sigma3': 1,
    'N_sim': 200
}

Data loaded, shape: (2400, 12)


In [3]:
# Function to compute true shares and derivatives
def compute_true_shares_and_derivatives(prices, x, xi, satellite, wired, params):
    J = len(prices)
    shares_sum = np.zeros(J)
    dsdp_sum = np.zeros((J, J))
    for i in range(params['N_sim']):
        beta_2 = np.random.normal(params['beta2_mean'], params['sigma2'])
        beta_3 = np.random.normal(params['beta3_mean'], params['sigma3'])
        V = params['beta1'] * x + params['alpha'] * prices + beta_2 * satellite + beta_3 * wired + xi
        V_max = np.max(V)
        exp_V = np.exp(V - V_max)
        exp_0 = np.exp(-V_max)
        denom = exp_0 + np.sum(exp_V)
        s_i = exp_V / denom
        shares_sum += s_i
        for j in range(J):
            for k in range(J):
                if j == k:
                    dsdp_sum[j, k] += params['alpha'] * s_i[j] * (1 - s_i[j])
                else:
                    dsdp_sum[j, k] += -params['alpha'] * s_i[j] * s_i[k]
    shares = shares_sum / params['N_sim']
    dsdp = dsdp_sum / params['N_sim']
    return shares, dsdp

In [29]:
# Compute true elasticities and diversion
true_elasticities = []
true_diversion = []
for t in range(1, T+1):
    indices = data['market_ids'] == t
    prices_t = data.loc[indices, 'prices'].values
    x_t = data.loc[indices, 'x'].values
    xi_t = data.loc[indices, 'xi_demand_unobs'].values
    satellite_t = data.loc[indices, 'satellite'].values
    wired_t = data.loc[indices, 'wired'].values
    shares_t, dsdp_t = compute_true_shares_and_derivatives(prices_t, x_t, xi_t, satellite_t, wired_t, true_params)
    elasticity_t = dsdp_t * prices_t[:, np.newaxis] / shares_t[:, np.newaxis]
    diversion_t = -dsdp_t / np.diag(dsdp_t)[:, np.newaxis]
    np.fill_diagonal(diversion_t, 1)
    true_elasticities.append(elasticity_t)
    true_diversion.append(diversion_t)
true_elasticities_avg = np.mean(true_elasticities, axis=0)
true_diversion_avg = np.mean(true_diversion, axis=0)
print("True Own-Price Elasticities:")
print(np.diag(true_elasticities_avg))
print("True Diversion Ratios:")
print(true_diversion_avg)

True Own-Price Elasticities:
[-3.71732887 -3.8371477  -3.88971357 -3.8063005 ]
True Diversion Ratios:
[[1.         0.32606781 0.2083313  0.2147882 ]
 [0.34656407 1.         0.20449473 0.20340677]
 [0.22314915 0.20734876 1.         0.32454543]
 [0.23170428 0.20681151 0.31908002 1.        ]]


In [31]:
# Set up for estimation
data['log_share'] = np.log(data['shares'])
data['outside_share'] = 1 - data.groupby('market_ids')['shares'].transform('sum')
data['logit_delta'] = data['log_share'] - np.log(data['outside_share'])
data['sum_other_x'] = data.groupby('market_ids')['x'].transform('sum') - data['x']
data['sum_other_w'] = data.groupby('market_ids')['w'].transform('sum') - data['w']

In [32]:
# OLS logit
X_ols = sm.add_constant(data[['prices', 'x', 'satellite', 'wired']])
y_ols = data['logit_delta']
ols_results = sm.OLS(y_ols, X_ols).fit()
print("OLS Logit Results:")
print(ols_results.summary())

OLS Logit Results:
                            OLS Regression Results                            
Dep. Variable:            logit_delta   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     663.9
Date:                Fri, 26 Sep 2025   Prob (F-statistic):          4.31e-314
Time:                        20:26:33   Log-Likelihood:                -3254.6
No. Observations:                2400   AIC:                             6517.
Df Residuals:                    2396   BIC:                             6540.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.459e+12   3.44e+12 

In [33]:
# IV logit
Z_iv = sm.add_constant(data[['x', 'w', 'sum_other_x', 'sum_other_w']])
X_iv = sm.add_constant(data[['prices', 'x', 'satellite', 'wired']])
iv_results = IV2SLS(y_ols, X_iv, Z_iv).fit()
print("IV Logit Results:")
print(iv_results.summary())

IV Logit Results:
                          IV2SLS Regression Results                           
Dep. Variable:            logit_delta   R-squared:                       0.125
Model:                         IV2SLS   Adj. R-squared:                  0.123
Method:                     Two Stage   F-statistic:                   -0.6817
                        Least Squares   Prob (F-statistic):               1.00
Date:                Fri, 26 Sep 2025                                         
Time:                        20:26:35                                         
No. Observations:                2400                                         
Df Residuals:                    2395                                         
Df Model:                           4                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -13.4347        nan  

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1884: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))


In [34]:
# Nested logit
data['nesting_ids'] = data['satellite'] * 1 + data['wired'] * 2
nl_formulation = pyblp.Formulation('prices + x + satellite + wired')
nl_problem = pyblp.Problem(nl_formulation, data)
nl_results = nl_problem.solve(rho=0.5)
print("Nested Logit Results:")
print(nl_results)
nl_elasticities = nl_results.compute_elasticities()
nl_diversion = nl_results.compute_diversion_ratios()
nl_elasticities_avg = nl_elasticities.reshape((T, J, J)).mean(axis=0)
nl_diversion_avg = nl_diversion.reshape((T, J, J)).mean(axis=0)
print("Nested Logit Own-Price Elasticities:")
print(np.diag(nl_elasticities_avg))
print("Nested Logit Diversion Ratios:")
print(nl_diversion_avg)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
600  2400   4    5     4     2 

Formulations:
     Column Indices:         0     1      2       3        4  
--------------------------  ---  ------  ---  ---------  -----
X1: Linear Characteristics   1   prices   x   satellite  wired
Solving the problem ...

Rho Initial Values:
 All Groups  
-------------
+5.000000E-01

Rho Lower Bounds:
 All Groups  
-------------
+0.000000E+00

Rho Upper Bounds:
 All Groups  
-------------
+9.900000E-01

Starting optimization ...



Detected collinearity issues with [wired] and at least one other column in X1. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
Detected collinearity issues with [wired] and at least one other column in ZD. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
The model may be under-identified. The total number of unfixed parameters is 6, which is more than the total number of moments, 4. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.
Detected that the 2SLS weighting matrix is nearly singular with condition number +2.411043E+17. To disable singularity checks, set options.singular_tol = numpy.inf.


GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm      Theta    
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1     00:00:00         0             1            0            0          0     +3.036718E+02                 +7.071669E+03  +5.000000E-01
 1     00:00:00         0             2            0            0          0     -1.720241E+02  +4.756960E+02  +5.323574E+03  +0.000000E+00
 1     00:00:00         1             3            0            0          0     +2.033948E-01                 +1.211885E-10  +2.147426E-01
 1     00:00:00         1             4            0            0          0     +6.256407E+02                 +3.959927E+03  +5.500686E-02
 1     00:00:00     

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.800764E+16. To disable singularity checks, set options.singular_tol = numpy.inf.



Failed to compute standard errors because of invalid estimated covariances of GMM parameters.

Computed results after 00:00:01.

Problem Results Summary:
GMM     Objective      Projected       Reduced     Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm     Hessian     Shares   Condition Number  Condition Number 
----  -------------  -------------  -------------  -------  ----------------  -----------------
 1    -1.720241E+02  +5.323574E+03  +0.000000E+00     0      +7.418157E+16      +2.360740E+02  

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm      Theta    
----  -----------  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 2     00:00:

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +3.892119E+16. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +3.892119E+16. To disable singularity checks, set options.singular_tol = numpy.inf.


In [ ]:
# BLP joint with optimal IV
blp_formulations = (
    pyblp.Formulation('prices + x + satellite + wired'),
    pyblp.Formulation('0 + satellite + wired'),
    pyblp.Formulation('w')
)
integration = pyblp.Integration('monte_carlo', size=50)
blp_problem = pyblp.Problem(blp_formulations, data, integration=integration, costs_type='log')
sigma0 = np.array([[1, 0], [0, 1]])
beta0 = np.array([0, -2, 1, 4, 4])
gamma0 = np.array([0.5, 0.25])
blp_results = blp_problem.solve(sigma=sigma0, beta=beta0, gamma=gamma0, rho=0, method='2s')
print("BLP Joint with Optimal IV Results:")
print(blp_results)
elasticities_est = blp_results.compute_elasticities()
diversion_ratios_est = blp_results.compute_diversion_ratios()
elasticities_avg = elasticities_est.reshape((T, J, J)).mean(axis=0)
diversion_ratios_avg = diversion_ratios_est.reshape((T, J, J)).mean(axis=0)
print("BLP Own-Price Elasticities:")
print(np.diag(elasticities_avg))
print("BLP Diversion Ratios:")
print(diversion_ratios_avg)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I     K1    K2    K3    MD    MS    H 
---  ----  ---  -----  ----  ----  ----  ----  ----  ---
600  2400   4   30000   5     2     2     4     2     2 

Formulations:
       Column Indices:             0        1      2       3        4  
-----------------------------  ---------  ------  ---  ---------  -----
 X1: Linear Characteristics        1      prices   x   satellite  wired
X2: Nonlinear Characteristics  satellite  wired                        
X3: Log Cost Characteristics       1        w                          
Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +1.000000E+00               
  wired    +0.000000E+00  +1.000000E+00

Rho Initial Values:
 All Groups  
-------------
+0.000000E+00

Beta Initial Values:
      1           prices            x          satellite        

Detected collinearity issues with [wired] and at least one other column in X1. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
Detected collinearity issues with [wired] and at least one other column in ZD. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
The model may be under-identified. The total number of unfixed parameters is 8, which is more than the total number of moments, 6. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.
Detected that the 2SLS weighting matrix is nearly singular with condition number +2.411043E+17. To disable singularity checks, set options.singular_tol = numpy.inf.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Encountered nonpositive marginal costs in a log-linear specification. This problem can sometimes be mitigated by bounding costs from below, choosing more reasonable initial parameter values, setting more conservative parameter bounds, or using a linear costs specification.
Reverted problematic marginal costs. Number of reverted elements: 38 out of 2400.

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                                                                                                          
Step     Time       Iterations   Evaluations  Iterations   Evaluatio

In [ ]:
# Merger simulations
data['firm_ids'] = data['firm_ids']
original_prices = data.groupby('product_ids')['prices'].mean().values

# Merger 1: firms 1 and 2
data_merger1 = data.copy()
data_merger1['merger_ids'] = data_merger1['firm_ids'].replace({1: 0, 2: 0, 3: 3, 4: 4})
changed_prices1 = blp_results.compute_prices(data_merger1, data_merger1['merger_ids'])
avg_prices1 = changed_prices1.reshape((T, J)).mean(axis=0)
print("Price Changes (1 and 2 merge):")
print(avg_prices1 - original_prices)

# Merger 2: firms 1 and 3
data_merger2 = data.copy()
data_merger2['merger_ids'] = data_merger2['firm_ids'].replace({1: 0, 3: 0, 2: 2, 4: 4})
changed_prices2 = blp_results.compute_prices(data_merger2, data_merger2['merger_ids'])
avg_prices2 = changed_prices2.reshape((T, J)).mean(axis=0)
print("Price Changes (1 and 3 merge):")
print(avg_prices2 - original_prices)

In [ ]:
# Merger with efficiencies
costs = blp_results.compute_costs()
merger_costs = costs.copy()
merger_costs[data_merger1['merger_ids'] == 0] *= 0.85
changed_prices_eff = blp_results.compute_prices(data_merger1, data_merger1['merger_ids'], costs=merger_costs)
avg_prices_eff = changed_prices_eff.reshape((T, J)).mean(axis=0)
print("Price Changes with 15% Cost Reduction:")
print(avg_prices_eff - original_prices)

# Welfare
cs_pre = blp_results.compute_consumer_surpluses()
cs_post = blp_results.compute_consumer_surpluses(prices=changed_prices_eff)
print("Change in Consumer Surplus:", (cs_post - cs_pre).mean())
profits_pre = blp_results.compute_profits()
profits_post = blp_results.compute_profits(prices=changed_prices_eff, costs=merger_costs)
print("Change in Total Profits:", (profits_post.sum(axis=1) - profits_pre.sum(axis=1)).mean())
print("Change in Total Welfare:", (cs_post - cs_pre + profits_post.sum(axis=1) - profits_pre.sum(axis=1)).mean())

## Results and Discussion

This notebook has completed all parts of the homework:

- Computed true elasticities and diversion ratios
- Estimated OLS and IV logit models
- Estimated nested logit model with elasticities and diversion
- Estimated BLP model with optimal IV
- Performed merger simulations for two scenarios
- Included efficiencies and welfare calculations

For the write-up, compile the printed outputs into tables as required.

The nested logit is misspecified because it assumes the random coefficients are correlated within nests, but in the true model, they are independent.

The merger between same-nest firms (1 and 2, both satellite) leads to higher price increases than merging across nests (1 and 3, satellite and wired).

Efficiencies can make the merger welfare-enhancing by reducing costs and thus prices.